### Product Analysis – costemic e-commerce data
Vladislav Ankushev; Matricul. №: 65346808

---
### Data set description
- Dataset contains behavior information for 5 months (Oct 2019 – Feb 2020) from a medium cosmetics online store.
- Each row in the file represents an event
- All events are related to products and users
- Each event is like many-to-many relation between products and user
- [Kaggle link](https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop)

### Main Task
- Let's pretend that the owner of a cosmetic website came to us and asked us to make a performance assessment of his product.
- We should pick several metrics to evaluate his website and give recommendations on what steps should be taken in the future.
- Every question is related to a concrete KPI.
- The recommendations should be general and include a 'call to action'.

### List of questions: 

**Conversion Rate**  
- What is the overall conversion rate? 
- Which year / month have the highest CR? 
- Which funnel steps should be improved?

<br>

**Retention**  
- What is the retention of the product?
- Do the RT values differ across different years and months?
- Are the RT values stable or not?

---
### (Spoiler) General Recommendation

**Conclusion:**
1. The overall CR values are low: only 7% of users make a purchase after viewing the product.
2. These values do not change radically if we look through particular years and months.
3. Therefore, we selected a subset of users who have been active for more than 14 days in the last year and month.
4. The results of this group were higher compared to the whole sample.
5. Consequently, we proposed that the main problem is related to retention.
6. We made an RT analysis and discovered that the product does not retain users well (there's no plateau).

<br>

**Recommendation:**
If the main problem is related to retention, then the product should consider using:
- push notifications;
- re-targeting campaigns;
- discounts & special offer days.

---
#### Libraries

In [1]:
import pandas as pd 
import warnings
import plotly.express as px
from datetime import datetime
import numpy as np
import plotly.graph_objects as go

warnings.filterwarnings('ignore')

### Data preprocessing

#### Read files
**Description:** using for-loop read the csv files from the main folder

In [2]:
# Define csv names
file_list = ['2019-Dec.csv', '2019-Nov.csv', '2019-Oct.csv', '2020-Feb.csv', '2020-Jan.csv']

# Create empty df
main_df = pd.DataFrame() 

# Iterate csv from folder
for i in file_list: 
    data = pd.read_csv('Datasets/' + i) 
    df = pd.DataFrame(data) 
    main_df = pd.concat([main_df,df],axis=0) 

main_df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5
...,...,...,...,...,...,...,...,...,...
4264747,2020-01-31 23:59:52 UTC,remove_from_cart,5886774,1487580006317032337,NaN,NaN,1.59,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47
4264748,2020-01-31 23:59:52 UTC,remove_from_cart,5886774,1487580006317032337,NaN,NaN,1.59,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47
4264749,2020-01-31 23:59:53 UTC,view,5875432,2084144451428549153,NaN,NaN,2.05,423651741,fb42a963-abef-4c4f-b1ba-f5812dd54e80
4264750,2020-01-31 23:59:57 UTC,remove_from_cart,5820745,1487580006317032337,NaN,NaN,2.22,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47


#### Change data types & cleaning

**Description:**
- Convert timestamp column to datetime (did not put it in for-loop, since it's slow)
- Create action_id column
- Create year, month, year_month column

In [3]:
# Convert to datetime 
main_df['event_time'] = pd.to_datetime(main_df['event_time'])

# Create action_id
main_df['action_id'] = main_df['event_time'].astype('str') + main_df['user_id'].astype('str') + main_df['user_session'].astype('str') 
main_df['action_id'] = main_df['action_id'] + main_df['product_id'].astype('str') + main_df['category_id'].astype('str')

# Create year, month, year_month column 
main_df['year'] = main_df['event_time'].dt.year
main_df['month'] = main_df['event_time'].dt.month
main_df['year_month'] = main_df['year'].astype('str') + ' - ' + main_df['month'].astype('str')

#### Double check data
**Description:** check whether every year + month were presented in main_df

In [4]:
main_df[['year', 'month']].value_counts().reset_index().sort_values(['year', 'month'])

,year,month,count
3,2019,10,4102283
0,2019,11,4635837
4,2019,12,3533286
1,2020,1,4264752
2,2020,2,4156682


---
### Data Analysis – Conversion Rate

**Questions:**
- What is the overall conversion rate? 
- Which year / month have the highest CR? 
- Which funnel steps should be improved?

<br>

**Analytical process:**
1. Calculate overall CR by users / actions 
2. Calculate CR by year_month (users / actions)
3. Calculate CR by year_month on concrete sample: 2020-02 (lifetime >= 14 days)

#### Calculation Overall – CR by users

In [5]:
# Num of users: view, cart, purchase
view_users = main_df.query('event_type == "view"').user_id.nunique()
cart_users = main_df.query('event_type == "cart"').user_id.nunique()
purchase_users = main_df.query('event_type == "purchase"').user_id.nunique()

# Calculate conversion steps
cr_view_cart_users = round(cart_users / view_users * 100, 2)
cr_cart_purch_users = round(purchase_users / cart_users * 100, 2)
cr_view_purch_users = round(purchase_users / view_users * 100, 2)

# Create a df
cr_df = pd.DataFrame({
    'cr_view_cart': [cr_view_cart_users],
    'cr_cart_purch': [cr_cart_purch_users],
    'cr_view_purch': [cr_view_purch_users]
})

# Create a categorical column
cr_df['type'] = 'users'

cr_df

,cr_view_cart,cr_cart_purch,cr_view_purch,type
0,24.93,27.75,6.92,users


#### Calculation – Overall CR by actions

In [6]:
# Num of actions: view, cart, purchase
view_actions = main_df.query('event_type == "view"').action_id.nunique()
cart_actions = main_df.query('event_type == "cart"').action_id.nunique()
purchase_actions = main_df.query('event_type == "purchase"').action_id.nunique()

# Calculate conversion steps
cr_view_cart_actions= round(cart_actions / view_actions * 100, 2)
cr_cart_purch_actions = round(purchase_actions / cart_actions * 100, 2)
cr_view_purch_actions = round(purchase_actions / view_actions * 100, 2)

# Create a df
cr_df_2 = pd.DataFrame({
    'cr_view_cart': [cr_view_cart_actions],
    'cr_cart_purch': [cr_cart_purch_actions],
    'cr_view_purch': [cr_view_purch_actions]
})

# Create a categorical column
cr_df_2['type'] = 'actions'

# Link frames
cr = pd.concat([cr_df, cr_df_2], axis=0, ignore_index=True)

cr

,cr_view_cart,cr_cart_purch,cr_view_purch,type
0,24.93,27.75,6.92,users
1,58.54,22.75,13.32,actions


#### Visualization – Overall CR by users (Sankey diagramm)

In [7]:
# Sankey diagramm

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label= ['View', 'Cart', 'Purchase']),
    link=dict(
        source = [0, 0, 1],  
        target = [1, 2, 2],  
        value = [value[0] for i, value in cr.query('type == "users"').iloc[:, :-1].items()],
    )
)])

fig.update_layout(title_text="<b> Sankey diagramm – Overall Conversion Rate by users" ,
                  font_size=10)
fig.show()

#### Visualization – Overall CR by actions (Sankey diagramm)

In [8]:
# Sankey diagramm
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label= ['View', 'Cart', 'Purchase']),
    link=dict(
        source = [0, 0, 1],  
        target = [1, 2, 2],  # индексы узлов назначений
        value = [value[1] for i, value in cr.query('type == "actions"').iloc[:, :-1].items()],
    )
)])

fig.update_layout(title_text="<b>  Sankey diagramm – Overall Conversion Rate by actions" ,
                  font_size=10)
fig.show()

### Visualization – Overall CR (Barplot)

In [9]:
# Convert to long format
long_cr = pd.melt(cr, id_vars=['type'], value_vars=['cr_view_cart', 'cr_cart_purch', 'cr_view_purch'])

# Barplot
fig = px.bar(long_cr, x='variable', y = 'value', color = 'type',
                   title='<b>Overall Conversion Rate</b>', barmode = 'group', text = 'value')

fig.update_layout(xaxis_title='CR steps', yaxis_title='CR values')

fig.show()

#### Interpretation – Overall CR

**Graph & CR description:**
- Graphs above illustrate funnel steps.
- User's flow: view product -> add item to the cart -> purchase item.
- Every conversion rate is relative (% from previous to current stage, e.g., from view product to add item to cart).
- There's one CR, which is absolute (% from first step to current stage, e.g., from view product to purchase).

<br>

**Interpretation:**
- The problematic stage is purchase.
- Few people add an item to the cart (25%), but even fewer users buy them (28%) – among those who added it to the cart.
- If we look at actions, we see that more than a half of actions belongs to adding an item to the cart (59%). But, after that, less than a quarter of actions belongs to purchase

---
#### Calculation CR by year (users)

In [10]:
# Num of users: view, cart, purchase
view_users = main_df.query('event_type == "view"').groupby('year_month').user_id.nunique().reset_index().rename(columns = {'user_id' : 'view'})
cart_users = main_df.query('event_type == "cart"').groupby('year_month').user_id.nunique().reset_index().rename(columns = {'user_id' : 'cart'})
purchase_users = main_df.query('event_type == "purchase"').groupby('year_month').user_id.nunique().reset_index().rename(columns = {'user_id' : 'purchase'})

# Link frames
all_users = view_users.merge(cart_users, on = 'year_month', how = 'left').merge(purchase_users, on = 'year_month', how = 'left')

# Calculate conversion steps
all_users['cr_view_cart'] = round(all_users['cart'] / all_users['view'] * 100, 2)
all_users['cr_cart_purch'] = round(all_users['purchase'] / all_users['cart'] * 100, 2)
all_users['cr_view_purch'] = round(all_users['purchase'] / all_users['view'] * 100, 2)

# Create a categorical column
all_users['type'] = 'users'

all_users

,year_month,view,cart,purchase,cr_view_cart,cr_cart_purch,cr_view_purch,type
0,2019 - 10,388331,133818,25762,34.46,19.25,6.63,users
1,2019 - 11,355643,95939,31524,26.98,32.86,8.86,users
2,2019 - 12,358212,83458,25613,23.30,30.69,7.15,users
3,2020 - 1,397775,92653,28220,23.29,30.46,7.09,users
4,2020 - 2,379246,89269,25759,23.54,28.86,6.79,users


#### Calculation CR by year (actions)

In [11]:
# Num of actions: view, cart, purchase
view_actions = main_df.query('event_type == "view"').groupby('year_month').action_id.nunique().reset_index().rename(columns = {'action_id' : 'view'})
cart_actions = main_df.query('event_type == "cart"').groupby('year_month').action_id.nunique().reset_index().rename(columns = {'action_id' : 'cart'})
purchase_actions = main_df.query('event_type == "purchase"').groupby('year_month').action_id.nunique().reset_index().rename(columns = {'action_id' : 'purchase'})

# Link frames
all_actions = view_actions.merge(cart_actions, on = 'year_month', how = 'left').merge(purchase_actions, on = 'year_month', how = 'left')

# Calculate conversion steps
all_actions['cr_view_cart'] = round(all_actions['cart'] / all_actions['view'] * 100, 2)
all_actions['cr_cart_purch'] = round(all_actions['purchase'] / all_actions['cart'] * 100, 2)
all_actions['cr_view_purch'] = round(all_actions['purchase'] / all_actions['view'] * 100, 2)

# Create a categorical column
all_actions['type'] = 'actions'

# Link final frames
all_cr = pd.concat([all_users, all_actions], axis = 0)

all_cr

,year_month,view,cart,purchase,cr_view_cart,cr_cart_purch,cr_view_purch,type
0,2019 - 10,388331,133818,25762,34.46,19.25,6.63,users
1,2019 - 11,355643,95939,31524,26.98,32.86,8.86,users
2,2019 - 12,358212,83458,25613,23.30,30.69,7.15,users
3,2020 - 1,397775,92653,28220,23.29,30.46,7.09,users
4,2020 - 2,379246,89269,25759,23.54,28.86,6.79,users
0,2019 - 10,1862050,1204218,245281,64.67,20.37,13.17,actions
1,2019 - 11,2075993,1289632,322269,62.12,24.99,15.52,actions
2,2019 - 12,1728221,911641,213066,52.75,23.37,12.33,actions
3,2020 - 1,2037501,1128662,263605,55.39,23.36,12.94,actions
4,2020 - 2,1953025,1118838,241881,57.29,21.62,12.38,actions


### Visualization – CR by year (Barplot)

In [12]:
# Convert to long format
long_all_cr = pd.melt(all_cr[['year_month', 'cr_view_cart', 'cr_cart_purch', 'cr_view_purch', 'type']], id_vars=['year_month', 'type'], 
                  value_vars=['cr_view_cart', 'cr_cart_purch', 'cr_view_purch'])
# Barplot
fig = px.bar(long_all_cr, x='year_month', y = 'value', color = 'variable',
             title='<b>Conversion rate by year & month groups</b>', barmode = 'group', text = 'value',
             facet_row = 'type')

fig.update_layout(xaxis_title='CR steps', yaxis_title='CR values', height = 500)

fig.show()

#### Interpretation – CR by year
- This graph illustrates conversion rates for different year-month subsets.
- As we can observe, there's no significant difference between groups. Except for 2019-10, where CR from cart to purch is the lowest by users.
- Probably, the seasonal factor does not affect users' behavior in buying cosmetics.
- And probably, the 2019-10 is the starting period, where less people bought any products

---
#### CR by year_month on concrete sample: 2020-02 (lifetime >= 14 days)

**Define lifetime**

In [13]:
# Query needed year & month & view
sample_2020_2_view = main_df.query('year_month == "2020 - 2" & event_type == "view"')

# Define min view time
min_view_time = sample_2020_2_view.groupby('user_id').event_time.min().reset_index().rename(columns= {'event_time':'min_view_time'})

# Merge with first df
sample_2020_2_view = sample_2020_2_view.merge(min_view_time, on = 'user_id', how = 'left')

# Calculate lt 
sample_2020_2_view['lt'] = (sample_2020_2_view['event_time'] - sample_2020_2_view['min_view_time']).dt.days

# Define users with >= 14 lt
long_lt_users = sample_2020_2_view.query('lt >= 14').user_id.unique()

# Filter needed users
filtered_users = main_df.query('year_month == "2020 - 2"')[main_df.query('year_month == "2020 - 2"')['user_id'].isin(long_lt_users)]

**Calculate CR** 

In [14]:
# Num of users / actions: view, cart, purchase
view_users_lt = filtered_users.query('event_type == "view"').user_id.nunique()
cart_users_lt = filtered_users.query('event_type == "cart"').user_id.nunique()
purchase_users_lt = filtered_users.query('event_type == "purchase"').user_id.nunique()

view_actions_lt = filtered_users.query('event_type == "view"').action_id.nunique()
cart_actions_lt = filtered_users.query('event_type == "cart"').action_id.nunique()
purchase_actions_lt = filtered_users.query('event_type == "purchase"').action_id.nunique()

# Calculate conversion steps
cr_view_cart_users_lt = round(cart_users_lt / view_users_lt * 100, 2)
cr_cart_purch_users_lt = round(purchase_users_lt / cart_users_lt * 100, 2)
cr_view_purch_users_lt = round(purchase_users_lt / view_users_lt * 100, 2)

cr_view_cart_actions_lt = round(cart_actions_lt / view_actions_lt * 100, 2)
cr_cart_purch_actions_lt = round(purchase_actions_lt / cart_actions_lt * 100, 2)
cr_view_purch_actions_lt = round(purchase_actions_lt / view_actions_lt * 100, 2)

# Create a df
cr_df_lt = pd.DataFrame({
    'cr_view_cart_lt': [cr_view_cart_users_lt],
    'cr_cart_purch_lt': [cr_cart_purch_users_lt],
    'cr_view_purch_lt': [cr_view_purch_users_lt]
})

cr_df_2_lt = pd.DataFrame({
    'cr_view_cart_lt': [cr_view_cart_actions_lt],
    'cr_cart_purch_lt': [cr_cart_purch_actions_lt],
    'cr_view_purch_lt': [cr_view_purch_actions_lt]
})

# Create a categorical column
cr_df_lt['type'] = 'users'
cr_df_2_lt['type'] = 'actions'

# Link frames
cr_lt = pd.concat([cr_df_lt, cr_df_2_lt], axis=0, ignore_index=True)

cr_lt

,cr_view_cart_lt,cr_cart_purch_lt,cr_view_purch_lt,type
0,59.22,45.62,27.01,users
1,58.15,21.88,12.72,actions


**Visualization – CR for 2020-02 & LT >= 14** 

In [15]:
# Convert to long format
long_cr_lt = pd.melt(cr_lt, id_vars=['type'], value_vars=['cr_view_cart_lt', 'cr_cart_purch_lt', 'cr_view_purch_lt'])

# Barplot
fig = px.bar(long_cr_lt, x='variable', y = 'value', color = 'type',
             title='<b>Conversion Rate for 2020-02 & LT >= 14</b>', barmode = 'group', text = 'value')

fig.update_layout(xaxis_title='CR steps', yaxis_title='CR values')

fig.show()

#### Interpretation CR 2020-02 & LT >= 14
- This graph illustrates conversion rates for users who have used the product for more than 14 days.
- As we can see, the conversion rate values, divided by users, have increased compared to the results for the whole group.
- We can say that practically every second person makes at least one transaction after putting an item in the cart (60% CR cart -> purchase).
- Every fifth action after putting an item in the cart corresponds to a purchase (22% CR cart -> purchase).
- Every fourth user, after viewing an item, will buy it (27% CR view -> purchase).
- Every tenth action after viewing an item corresponds to a purchase (13% CR view -> purchase).


---
### Data Analysis – Retention

**Questions:**
- What is the retention of the product?
- Do the RT values differ across different years and months?
- Are the RT values stable or not?

#### Overall RT calculation

In [16]:
# Query view actions
view_data = main_df.query('event_type == "view"')

# Define min view time
min_view_time = view_data.groupby('user_id').event_time.min().reset_index().rename(columns= {'event_time':'min_view_time'})

# Merge with first df
view_data = view_data.merge(min_view_time, on = 'user_id', how = 'left')

# Calculate lt 
view_data['lt'] = (view_data['event_time'] - view_data['min_view_time']).dt.days

# Calculate RT 
rt_table = view_data.groupby('lt').user_id.nunique().reset_index()

# Filter: take first 30 days
rt_table = rt_table.query('lt <= 30')

# Define num of users on 1st day
rt_table['first_day'] = rt_table.iloc[:1 :].user_id[0]

# Calculate RT
rt_table['rt'] = round(rt_table['user_id'] / rt_table['first_day'] * 100, 2)

# Filter: delete first day
rt_table = rt_table.query('lt >= 1')

#### Visualization - Overall RT

In [17]:
fig = px.line(rt_table, x="lt", y="rt",  
                                title='<b>Overall Retention</b>',
                                markers=True,
                                text="rt")
fig.update_layout(xaxis_title='Num of Return day', yaxis_title='Retention value', height=600)
fig.update_traces(textposition="top center")
fig.show()

#### Interpretation – Overall RT
- It is incorrect to calculate RT by view actions, since it is not the first action
- But, still we see that trend continue to decrease after 30 lifetime: we don't observe the plateau

---
#### RT by year & month calculation

In [18]:
# Calculate RT by year & month
rt_table_year_month = view_data.groupby(['lt', 'year_month']).user_id.nunique().reset_index()

# Filter: take first 30 days
rt_table_year_month = rt_table_year_month.query('lt <= 30')

# Calculate users for lt = 0 
first_day_users = rt_table_year_month[rt_table_year_month['lt'] == 0].set_index('year_month')['user_id']
rt_table_year_month['first_day'] = rt_table_year_month['year_month'].map(first_day_users)

# Calculate RT 
rt_table_year_month['rt'] = round(rt_table_year_month['user_id'] / rt_table_year_month['first_day'] * 100, 2)

# Filter: delete first day
rt_table_year_month = rt_table_year_month.query('lt >= 1')

In [19]:
rt_table_year_month

,lt,year_month,user_id,first_day,rt
5,1,2019 - 10,15606,388331,4.02
6,1,2019 - 11,11962,305079,3.92
7,1,2019 - 12,10417,292312,3.56
8,1,2020 - 1,11850,321500,3.69
9,1,2020 - 2,10496,291847,3.60
...,...,...,...,...,...
150,30,2019 - 10,303,388331,0.08
151,30,2019 - 11,3877,305079,1.27
152,30,2019 - 12,1847,292312,0.63
153,30,2020 - 1,1485,321500,0.46


#### Visualization - RT by year & month

In [20]:
fig = px.line(rt_table_year_month, x="lt", y="rt",  
                                title='<b>Retention by year & month</b>',
                                color = 'year_month')
fig.update_layout(xaxis_title='Num of Return day', yaxis_title='Retention value', height=600)
fig.update_traces(textposition="top center")
fig.show()

#### Interpretation - Retention by year & month
- Still the above graph is incorrect, since we use view actions to calculate RT 
- However, we can say, that in November 2019, users 'visited' website more often, than the other dates.
- We don't observe the plateau for each month-year
- Therefore, the website should probably focus on increasing Retention

---
### General Recommendation

**Conclusion:**
1. The overall CR values are low: only 7% of users make a purchase after viewing the product
2. These values don't change if we look through particular years and months
3. Therefore, we selected a sample of users who have been active for more than 14 days in the last year and month
4. The results of this group were higher compared to the whole sample
5. Consequently, we proposed that the main problem is related to retention
6. We made an RT analysis and discovered that the product does not retain users well (there's no plateau)

<br>

**Recommendation:**
If the main problem is related to retention, then the product should consider using:
- push notifications
- re-targeting campaigns
- discounts & special offer days

---
#### Export frames as csv for streamlit

In [21]:
cr.to_csv('cr.csv', index=False)
long_cr.to_csv('long_cr.csv', index=False)

all_cr.to_csv('all_cr.csv', index=False)
long_all_cr.to_csv('long_all_cr.csv', index=False)

cr_lt.to_csv('cr_lt.csv', index=False)
long_cr_lt.to_csv('long_cr_lt.csv', index=False)

rt_table.to_csv('rt_table.csv', index=False)
rt_table_year_month.to_csv('rt_table_year_month.csv', index=False)